In [128]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import warnings


In [129]:
url="https://www.amazon.in/s?k=bag&crids=DFPVAY6HSK3T&sprefix=bag%2Caps%2C250&ref=sr_pg_"
head_url="https://www.amazon.in"


In [130]:
headers=({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36 Edg/115.0.1901.203',"Accept-Language":'en-US,en;q=0.5'})

In [131]:
def creating(url,headers,head_url):
    columns=["Product_url","Product_name","Product_price","Rating","Number_reviews","Description","Product_Description","Manufecturer","ASIN"]
    df=pd.DataFrame(columns=columns)
    for x in range(21):
        x=x+1
        new_url=url+str(x)
        web=requests.get(new_url,headers=headers)
        soup=BeautifulSoup(web.content,"html.parser")
        links=soup.find_all("a",attrs={'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})
        
        for items in range(len(links)):
            
            product_url=head_url+links[items].get('href')
            new_web=requests.get(product_url,headers=headers)
            
            inner_soup=BeautifulSoup(new_web.content,"html.parser")
            try:
                name=inner_soup.find("h1",attrs={'id':'title'}).text
            except Exception as e:
                name="nil"
        
            try:
                product_rating=inner_soup.find("span",attrs={'class':'a-size-base a-color-base'}).text
                product_rating=float(product_rating.strip()[0])
            except Exception as e:
                product_rating=0
            try:
                product_price=inner_soup.find("span",attrs={'class':'a-price-whole'}).text
            except Exception as e:
                product_price="0"
            try:   
                product_review=inner_soup.find("span",attrs={'d':'acrCustomerReviewText'}).text
            except Exception as e:
                product_review="nil"
                
            try:    
                product_description=inner_soup.find("ul",attrs={'class':'a-unordered-list a-vertical a-spacing-mini'}).text
            except Exception as e:
                product_description="nil"
                
            try:
                description=inner_soup.find("ul",attrs={'class':'a-unordered-list a-vertical a-spacing-mini'}).text
            except Exception as e:
                description="nil"
                
            try:
                asin_pattern = r'B[A-Z0-9]{9}'
                asin=inner_soup.find('span',attrs={'class':'a-list-item'}).text
                asin_no=re.findall(asin_pattern, asin)
            except Exception as e:
                asin_no="nil"
            try:
                
                manufacturer_pattern = r'Manufacturer: (.+)'
                manufact=inner_soup.find('span',attrs={'class':'a-list-item'}).text
                manufacture=re.findall(manufacturer_pattern,manufact)
            except Exception as e:
                manufacture="nil"
                
            data={"Product_url":product_url,"Product_name":name,"Product_price":product_price,"Rating":product_rating,"Number_reviews":product_review,"Description":description,"Product_Description":product_description,"Manufecturer":manufacture,"ASIN":asin_no}
            df=df.append(data,ignore_index=True)
            return df
        
        
        
        

In [132]:
df=creating(url,headers,head_url)

In [133]:
df

,Product_url,Product_name,Product_price,Rating,Number_reviews,Description,Product_Description,Manufecturer,ASIN
0,https://www.amazon.in/uppercase-Professional-r...,nil,0,0,nil,nil,nil,nil,nil


In [ ]:
df.to_csv('output.csv', index=False)